In [ ]:
#! /usr/bin/python3.5
#https://learningtensorflow.com/ReadingFilesBasic/
#https://www.tensorflow.org/tutorials/using_gpu

# To compute term frequancy from multi txt file and save it in multi csv and one csv by parellel process using tensorflow on GPU


In [ ]:
import os

import tensorflow as tf
import csv
from tensorflow.python.client import timeline
import nltk
from nltk.wsd import lesk
from nltk.corpus import wordnet as wn
from collections import Counter

import pandas as pd
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.wsd import lesk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk import PorterStemmer
from nltk.stem.porter import *
from nltk.stem.snowball import SnowballStemmer
from string import digits
#from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer



# Configuration Data

## Pathes

### Data Source

In [ ]:
dir_path = os.path.dirname(os.path.realpath(__file__))

path_data_source=dir_path+"/data/data_source/"

#sub_path_data_source="demo_onefile/"

sub_path_data_source="demo/"

file_path=path_data_source+sub_path_data_source

file_names = [os.path.join(file_path, f) 
                      for f in os.listdir(file_path) 
                      if f.endswith(".txt")]

#sub_path_data_source="demo/"





### Table Database

In [ ]:

path_database=dir_path+"/data/database/csv/"
path_sub_tfidf="sub_tfidf/"
path_full_tfidf="full_tfidf/"
path_tf="sub_tf/"


## Data Source

In [ ]:
data_source = 'cs.txt'


## Tables

#### Table Data Base

In [ ]:
TF_File="TF-"
TF_Full="TF-Full.csv"


# Configuration session

In [ ]:
'''config = tf.ConfigProto(device_count={"CPU": 7},
                        allow_soft_placement=True,
                        inter_op_parallelism_threads=1,
                        intra_op_parallelism_threads=1,
                        use_per_session_threads=True)'''
#import pprint



config = tf.ConfigProto(device_count={"GPU": 2,"CPU":32},
                        allow_soft_placement=True,
                        inter_op_parallelism_threads=32,
                        intra_op_parallelism_threads=32,
                        use_per_session_threads=True,
                        log_device_placement=True)

#pprint.pprint({ k:str(getattr(config,k)) for k in sorted(dir(config)) if not k.startswith('_')})


# Generic function

## Write data to csv

In [ ]:
'''
Write Excell sheet
'''
def save_file_to_database(data_rows,path_database,file_databbase,header_list):
    outfile = open(path_database+file_databbase,'w')
    writer=csv.writer(outfile)
    #header_list=['uuid','paragraph','doc_id']
    i=0
    for line in data_rows:
        row=[i,line,'paragraph no.'+str(i)]
        if i==0:
            
            writer.writerow(header_list)
            writer.writerow(row)
        else:
            #print('ff')
            writer.writerow(row)
        i+= 1
        #outfile.close()
            

## Read data from csv

In [ ]:
'''
Read Excell sheet
'''
def read_text_from_database(path_database,file_databbase):
    queue_paragraph=[]
    #f = open(sys.argv[1], 'rt')
    outfile = open(path_database+file_databbase,'rt')
    try:
                
        reader=csv.reader(outfile)
        for row in reader:
            queue_paragraph.append(row)
            #print (row)
    finally:
        print ("row")
        outfile.close()
        
    return queue_paragraph
    

In [ ]:
def read_cvs_by_pands(path_database,file_databbase,index_col, header):
    return pd.read_csv(path_database+file_databbase,index_col=index_col,header=header)

#read_cvs_by_pands(path_database,paragraph_table,index_col=0,header=0)

In [ ]:
def write_cvs_by_pands(path_database,file_databbase,header,data_rows):
    csv_df=pd.DataFrame(data_rows,columns=header ) 
    csv_df.to_csv(path_database+file_databbase)

    

#write_cvs_by_pands(path_database,sentences_paragraph_table,sentences_paragraph_list,sales)

In [ ]:
#save pragraphs to files
def write_file(pragraph,num_pragraph,path):
    file = open(path+str(num_pragraph)+".txt","w") 
 
    file.write(pragraph) 
    
    file.close() 
    

In [ ]:
#create sub dataset
def sub_dataset(path_data_source,data_source):
    pragraphs=txt_pragraphs(read_file(path_data_source+data_source))
    counter=0
    for pragraph in pragraphs:
        print('pragraph no ',counter)
        write_file(pragraph,counter,sub_path_data_source)
        counter +=1
    

# Pre processing pipeline

## Read Huge File

In [ ]:
def read_file(str):
    file = open(str,'r')
    txt=file.read()
    #print(txt)
    return txt

## Split Document to pragraphs

In [ ]:
def txt_pragraphs(str):
    pragraphs = str.split("\n\n")
    return pragraphs
#pragraphs=txt_pragraphs(txt)
#type(pragraphs)

## Save paragraphs in excel 

In [ ]:
#save_file_to_database(txt_pragraphs(read_file(path_data_source+data_source)),path_database,paragraph_table,paragraph_header_list)

## Split Paragraph to Sentences

In [ ]:
def pragraph_to_setnences(str):
    return sent_tokenize(str)
#setnences=pragraph_to_setnences(pragraphs[n_pragraph])

## Word Process For Sentence

### Removing English stopwords and Punct per Sentence

In [ ]:
new_stop_words = ['the', 'that', 'to', 'as', 'there', 'has', 'and', 'or', 'is', 'not', 'a', 'of', 'but', 'in', 'by', 'on', 'are', 'it', 'if','what','where','how','when']
new_stop_words2=['--','i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'now','even','until','then','must']
numbers=[1,2,3,4,5,6,7,8,9]
#stemmer = SnowballStemmer("english", ignore_stopwords=True)
def remove_stopword_sentences(str):
    tokenizer = RegexpTokenizer("[\w']+")
    
    words=tokenizer.tokenize(str)
    
    english_stops = set(stopwords.words('english'))
    stems=[]
    list_word=[word for word in words if word.lower() not in english_stops and word.lower() not in new_stop_words and word.lower() not in new_stop_words2 and  not word.lower().isdigit() and word.lower() not in digits and word.lower() not in  numbers]
    
    for word in list_word:
        #stems.append(stem(word))
        #stems.append(PorterStemmer().stem(word))
        #stems.append(stemmer.stem(word))
        #stems.append(stemmer.stem("computer"))
        stems.append(word)
    
    return stems#(stem(setem_word for setem_word in  ([word for word in words if word not in english_stops and word not in new_stop_words])))

In [ ]:
def word_list_sentece(pragraph):
    words_list=[]
    setnences=pragraph_to_setnences(pragraph)
    for indexs in range(len(setnences)):    
        #print("Sentence No. ",indexs,": ",setnences[indexs],"\n")
        words=remove_stopword_sentences(setnences[indexs])
        wordsent=''
        for index in range(len(words)):
            wordsent+=' '+words[index]
            #print("wordsent:",wordsent)
            
        words_list.append(wordsent)
        #count = Counter(words)
        #print("wordsent:",wordsent)
        #print(" word:",words)
    print(words_list)
    return words_list

#corpus=word_list_sentece(pragraphs[0])

## Word Sense Disambiguation (WSD): LESK per Sentence

Given an ambiguous word and the context in which the word occurs, Lesk returns a Synset with the highest number of overlapping words between the context sentence and different definitions from each Synset.


In [ ]:

'''
this function for compute lesk for each word(list of word) in sentence
'''
def lesk_words_sentence(words,sentence):
    lesks= []
    for word in words:
        if lesk(sentence,word, 'n') is not None:
            lesks.append(lesk(sentence,word, 'n'))
            #print("Word is: ",word,"\n LESK: ",lesk(sentence,word, 'n'),"\n Sentence: ",sentence )
        
    return lesks

In [ ]:
'''
this function for compute lesk of word in sentence
'''

def lesk_word_sentence(sentence,word):
    from nltk.wsd import lesk
    lesk_synset=''
    #lesks= []
    #for word in words:
    #disambiguated=lesk(context_sentence=sentence, ambiguous_word=word)
    disambiguated=lesk(sentence,word, 'n')
    #print(disambiguated)
    #if disambiguated is not None:
    lesk_synset=disambiguated
    #else:
    #lesk_synset=0
    #print("Word is: ",word,"\n LESK: ",lesk(sentence,word, 'n'),"\n Sentence: ",sentence )
        
    return lesk_synset

#lesk("Computer science is a discipline that spans theory and practice","science")

#sent = 'people should be able to marry a person of their choice'.split()
#lesk(sent, 'able')

In [ ]:
def add_row_csv(path_database,idf,list_data):
    with open(path_database+idf, 'a') as f:
        writer = csv.writer(f)
        writer.writerow(list_data)

In [ ]:
def name_file(full_name_path):
    d=full_name_path.split("/")
    print(d)
    name=d[len(d)-1].split(".")
    return name[0]

In [ ]:
                
with tf.Session(config=config) as sess:
        index_paragraph=0
        col=1
                
        index_file=0
        sess.run(tf.global_variables_initializer())
        #print(file_names)
        
        for filename in file_names:
            #print("index_file",str(index_file))
            word_file_fatma=[]
            with open(filename) as inf:
                #print("tpe",type(inf))
                txt=inf.read()
                
                paragraph_list=txt_pragraphs(txt)   
        
        
                for paragraph in paragraph_list: #get pragraphs(documents) from DB
                    #print("Pragraph type ",type(paragraph))
                   
                    
                    if index_paragraph ==0:
                        index_paragraph += 1
                    else:
                        
                        setnences=pragraph_to_setnences(paragraph)#partitions paragraph to sentence
                        
                        
                        for setnence in setnences:
                            #print("  ",setnence)                            

                            words=remove_stopword_sentences(setnence)#remove stop words and noise

                            for word in words:

                                
                                lesk=lesk_word_sentence(setnence,word)#get LESK of word in sentence

                                #paragraph_word.append(word_sentence)

                                if lesk is not None:
                                    
                                   
                                    word_file_fatma.append(lesk.name())
                                

                            
                            '''////////////////END Sentence////////////////# '''
                            
                        
                       

                       
                        '''////////////////END PARAGRAPH////////////////# '''

                
                
                word_file_Freq=Counter(word_file_fatma)
                sum_count=sum(word_file_Freq.values())

                
                freq=[]
                for i in word_file_Freq.values():
                    c=i/sum_count
                    freq.append(c)
                csv_df=pd.DataFrame([freq],columns=word_file_Freq.keys() ) 
                
                
                csv_df.to_csv(path_database+path_tf+TF_File+name_file(filename)+".csv")
                # add to idf file 
                
                full_list=[]
                full_list.insert(0,name_file(filename))
                full_list=full_list+list(word_file_Freq.keys())
                # add to single
                
                add_row_csv(path_database+path_sub_tfidf,name_file(filename)+".csv",full_list)
                # add to total idf file 
                add_row_csv(path_database+path_full_tfidf,TF_Full,full_list)
                index_file +=1
                